In [2]:
import requests
import json
import urllib
import os 
import datetime as dt

api_key = '384d0e4a284d03c77763c19a39f2e039'

# 전달 요청 파라미터
q = '박민영'
page = 1
size =  80

# 폴더 이름 생성
datetime = dt.datetime.now().strftime('%y%m%d-%H%M%S')
dirname = '%s_%s' % (q, datetime)
dirname

'박민영_210520-102947'

In [3]:
if not os.path.exists(dirname):
    os.mkdir(dirname)

In [4]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"
header_info = {'User-agent': user_agent, 'referer': None, 'Authorization': 'KakaoAK ' + api_key}
header_info

{'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
 'referer': None,
 'Authorization': 'KakaoAK 384d0e4a284d03c77763c19a39f2e039'}

In [5]:
params = {'page':page, 'size':size,'query' : q}
query = urllib.parse.urlencode(params)
query

'page=1&size=80&query=%EB%B0%95%EB%AF%BC%EC%98%81'

In [8]:
url_tpl = 'https://dapi.kakao.com/v2/search/image'
api_url = url_tpl + '?' + query
api_url

'https://dapi.kakao.com/v2/search/image?page=1&size=80&query=%EB%B0%95%EB%AF%BC%EC%98%81'

In [9]:
r = requests.get(api_url, headers=header_info)

# 결과 검사
if r.status_code != 200:
    # 에러코드와 에러메시지를 문자열로 구성
    err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
    # 강제로 에러를 발생시킨다.
    raise Exception(err_msg)

r.encoding = 'utf-8'
image_dict = json.loads(r.text)
image_dict

collection': 'cafe',
   'datetime': '2020-09-02T11:20:20.000+09:00',
   'display_sitename': 'Daum카페',
   'doc_url': 'http://cafe.daum.net/altkfl/k9AT/11724',
   'height': 812,
   'image_url': 'https://img1.daumcdn.net/relay/cafe/original/?fname=https%3A%2F%2Fpds.joins.com%2Fnews%2Fcomponent%2Fhtmlphoto_mmdata%2F202002%2F04%2Fccdf3134-b4a5-4b06-823f-4de55b0cf405.jpg',
   'thumbnail_url': 'https://search2.kakaocdn.net/argon/130x130_85_c/EQ8JCahDYdH',
   'width': 559},
  {'collection': 'blog',
   'datetime': '2009-02-02T16:39:05.000+09:00',
   'display_sitename': 'Daum블로그',
   'doc_url': 'http://blog.daum.net/sanokl/13584559',
   'height': 887,
   'image_url': 'http://cfs8.blog.daum.net/image/1/blog/2008/11/02/19/53/490d86903df9f&filename=%EB%B0%95%EB%AF%BC%EC%98%814.jpg',
   'thumbnail_url': 'https://search4.kakaocdn.net/argon/130x130_85_c/7Ol1frXPNZp',
   'width': 480},
  {'collection': 'cafe',
   'datetime': '2010-01-16T08:55:32.000+09:00',
   'display_sitename': 'Daum카페',
   'doc_url'

In [11]:
image_list = image_dict['documents']

for i, v in enumerate(image_list):
    path = '%s/%s_%04d.jpg' %(dirname, q, i)
    print('[%s] >> %s' % (path, v['image_url']))

    header_info['referer'] = v['doc_url']

    r = requests.get(v['image_url'], headers=header_info, stream=True)

    if r.status_code != 200:
        # 에러코드와 에러메시지를 문자열로 구성
       err_msg = "%d %s 에러가 발생했습니다." % (r.status_code, r.reason)
       print(err_msg)
       continue

    try:
        # 추출한 데이터를 저장
        # -> 'w': 텍스트 쓰기 모드, 'wb': 바이너리(이진값) 쓰기 모드
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print("----------> 저장성공")
    except:
        print("~~~~~~~~~~~> 저장실패")
        continue

[박민영_210520-102947/박민영_0000.jpg] >> https://t1.daumcdn.net/news/202102/25/ilgansports/20210225113437503jwuu.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0001.jpg] >> https://t1.daumcdn.net/news/202102/25/ilgansports/20210225113436254rmym.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0002.jpg] >> https://t1.daumcdn.net/news/202102/25/ilgansports/20210225113434983fuha.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0003.jpg] >> https://t1.daumcdn.net/news/202104/16/ilgansports/20210416090434157zoie.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0004.jpg] >> https://t1.daumcdn.net/news/202102/17/ilgansports/20210217094008543sdfj.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0005.jpg] >> http://www.viva100.com/mnt/images/file/2016y/01m/20d/20160120001031453_2.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0006.jpg] >> https://t1.daumcdn.net/news/202102/17/ilgansports/20210217094009578zzfe.jpg
----------> 저장성공
[박민영_210520-102947/박민영_0007.jpg] >> https://img1.daumcdn.net/relay/cafe/original/?fname=http%3A%

In [12]:
is_end = False
page = 1

image_list = []

while not is_end:
    params = {'page':page, 'size':size, 'query':q}
    query = urllib.parse.urlencode(params)
    
    url_tpl = "https://dapi.kakao.com/v2/search/image"
    api_url = url_tpl + "?" + query
    api_url

    r = requests.get(api_url, headers=header_info)

    if r.status_code != 200:
        print("[%d Error] %s" % (r.status_code, r.reason))
        quit()

    r.encoding = "utf-8"
    image_dict = json.loads(r.text)
    image_dict
    
    is_end = image_dict['meta']['is_end']
    image_list += image_dict['documents']
    
    print("%dpage로부터 %d건 수집함 >>> 누적 데이터 수: %d" % (page, len(image_dict['documents']), len(image_list)))
    page += 1
    
    if page >= 50:
        is_end = True
    
image_list

1page로부터 80건 수집함 >>> 누적 데이터 수: 80
2page로부터 80건 수집함 >>> 누적 데이터 수: 160
3page로부터 80건 수집함 >>> 누적 데이터 수: 240
4page로부터 80건 수집함 >>> 누적 데이터 수: 320
5page로부터 80건 수집함 >>> 누적 데이터 수: 400
6page로부터 80건 수집함 >>> 누적 데이터 수: 480
7page로부터 80건 수집함 >>> 누적 데이터 수: 560
8page로부터 80건 수집함 >>> 누적 데이터 수: 640
9page로부터 80건 수집함 >>> 누적 데이터 수: 720
10page로부터 80건 수집함 >>> 누적 데이터 수: 800
11page로부터 80건 수집함 >>> 누적 데이터 수: 880
12page로부터 80건 수집함 >>> 누적 데이터 수: 960
13page로부터 80건 수집함 >>> 누적 데이터 수: 1040
14page로부터 80건 수집함 >>> 누적 데이터 수: 1120
15page로부터 80건 수집함 >>> 누적 데이터 수: 1200
16page로부터 80건 수집함 >>> 누적 데이터 수: 1280
17page로부터 80건 수집함 >>> 누적 데이터 수: 1360
18page로부터 80건 수집함 >>> 누적 데이터 수: 1440
19page로부터 80건 수집함 >>> 누적 데이터 수: 1520
20page로부터 80건 수집함 >>> 누적 데이터 수: 1600
21page로부터 80건 수집함 >>> 누적 데이터 수: 1680
22page로부터 21건 수집함 >>> 누적 데이터 수: 1701


 'https://search4.kakaocdn.net/argon/130x130_85_c/HajxzHgFSSZ',
  'width': 600},
 {'collection': 'blog',
  'datetime': '2021-03-09T23:45:09.000+09:00',
  'display_sitename': '티스토리',
  'doc_url': 'http://somlove.tistory.com/732',
  'height': 512,
  'image_url': 'https://k.kakaocdn.net/dn/w7vIn/btqZKAgQBsV/ZmUVfqd31pAezk3nEIrhMk/img.jpg',
  'thumbnail_url': 'https://search1.kakaocdn.net/argon/130x130_85_c/LX0o7OEyXpS',
  'width': 319},
 {'collection': 'blog',
  'datetime': '2009-11-12T12:41:00.000+09:00',
  'display_sitename': '네이버블로그',
  'doc_url': 'http://blog.naver.com/text33/60094894655',
  'height': 248,
  'image_url': 'http://postfiles2.naver.net/20091112_257/text33_1257996891824hjEb5_jpg/%EB%B0%95%EB%AF%BC%EC%98%81%EB%AF%B8%EB%8B%88%ED%99%88%ED%94%BC_text33.jpg?type=w2',
  'thumbnail_url': 'https://search1.kakaocdn.net/argon/130x130_85_c/3hfHhKMTpl1',
  'width': 510},
 {'collection': 'news',
  'datetime': '2016-04-04T20:05:02.000+09:00',
  'display_sitename': '마이데일리',
  'doc_url':

In [13]:
# 이미지 주소에 대해서만 반복
for i, v in enumerate(image_list): 
    image_url = v['image_url']
    
    # 파일이 저장될 경로 생성
    path = "%s/%s_%04d.jpg" % (dirname, q, i)
    print( "[%s] >> %s" % (path, image_url) )

    # 예외처리 구문 적용
    try:
        # 이미지 주소를 다운로드를 위해 stream 모드로 가져온다.
        r = requests.get(image_url, headers=header_info, stream=True)
    
        # 에러 발생시 저장이 불가능하므로 건너뛰고 반복의 조건식으로 이동
        if r.status_code != 200:
            print("##########> 저장실패 (%d)" % r.status_code)
            continue
        
        # 추출한 데이터를 저장
        # -> 'w': 텍스트 쓰기 모드, 'wb': 바이너리(이진값) 쓰기 모드
        with open(path, 'wb') as f:
            f.write(r.raw.read())
            print("----------> 저장성공")
    except:
        print("~~~~~~~~~~~> 저장실패")
        continue

ver.net/MjAyMDEyMDNfMTQ5/MDAxNjA2OTY1OTUxNzc0.1-O5hgCrD0rO3SbSvxBsJp4TmFI0oy1tepC-1yzoQuEg.BeGBkGA_OwMRruehPxK4A8vk-7sWx7H2jMIEQ9uMNVQg.JPEG.pamahaeyo/%EC%97%98%EB%A6%AC%EC%9E%90%EB%B2%B3%ED%8E%8C_%EB%B0%95%EB%AF%BC%EC%98%81.jpg?type=w966
----------> 저장성공
[박민영_210520-102947/박민영_1586.jpg] >> http://postfiles5.naver.net/20131025_292/rlawjdl90_1382678717587EtpCD_PNG/%B9%E8%BF%EC%B9%DA%B9%CE%BF%B5%C3%D6%B1%D9%C6%C4%B8%AE%B1%D9%C8%B2%BB%E7%C1%F8%B8%F0%C0%BD_%285%29.png?type=w2
----------> 저장성공
[박민영_210520-102947/박민영_1587.jpg] >> http://t1.daumcdn.net/cfile/tistory/1766684C4FF4EDB918
##########> 저장실패 (403)
[박민영_210520-102947/박민영_1588.jpg] >> https://img1.daumcdn.net/relay/cafe/original/?fname=https%3A%2F%2Fpds.joins.com%2Fnews%2Fcomponent%2Fhtmlphoto_mmdata%2F201908%2F20%2Fhtm_2019082083459468427.jpg
##########> 저장실패 (403)
[박민영_210520-102947/박민영_1589.jpg] >> http://postfiles6.naver.net/20150805_213/grace_woo_14387543007197iTUJ_JPEG/%B9%DA%B9%CE%BF%B54.jpg?type=w2
----------> 저장성공
[박민영_210520